In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [2]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
creds = ''
from pathlib import Path
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
path = Path('us-patent-phrase-to-phrase-matching')

In [4]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [6]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [5]:
df = pd.read_csv(path/'train.csv')

In [6]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [9]:
df.head()


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [10]:
df.describe(include='object')
# object means non numeric data types

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [45]:
df['input'] = 'TEXT1:' + df.context + '; TEXT2: ' + df.target +  '; ANC1: ' + df.anchor

In [46]:
df.input.head()


0    TEXT1:A47; TEXT2: abatement of pollution; ANC1: abatement
1            TEXT1:A47; TEXT2: act of abating; ANC1: abatement
2           TEXT1:A47; TEXT2: active catalyst; ANC1: abatement
3       TEXT1:A47; TEXT2: eliminating process; ANC1: abatement
4             TEXT1:A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [47]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [48]:
model_nm = 'microsoft/deberta-v3-small'

In [49]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from fastai.distributed import *
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [50]:
tokz.tokenize("Hello, my name is Rahul and I am a loser")

['▁Hello',
 ',',
 '▁my',
 '▁name',
 '▁is',
 '▁Rahul',
 '▁and',
 '▁I',
 '▁am',
 '▁a',
 '▁loser']

In [51]:
tokz("A platypus is an ornithorhynchus anatinus.")


{'input_ids': [1, 336, 114224, 269, 299, 289, 4840, 34765, 102530, 1867, 299, 2401, 26835, 260, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [52]:
def tok_func(x):
    return tokz(x["input"])

In [53]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [54]:
tokz.vocab['▁of']

265

In [55]:
tok_ds = tok_ds.rename_columns({'score': 'labels'})

In [56]:
eval_df = pd.read_csv(path/'test.csv')

In [57]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [58]:
dds = tok_ds.train_test_split(0.25, seed=42)
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)
from transformers import TrainingArguments,Trainer

  0%|          | 0/1 [00:00<?, ?ba/s]

In [59]:
bs = 128
epochs = 4

In [60]:
lr = 8e-5
def corr(x,y): return np.corrcoef(x,y)[0][1]
#takes a tuple with the first and second text
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}
from transformers import (AutoTokenizer, AutoConfig,
                              AutoModelForSequenceClassification, TrainingArguments, Trainer)

In [61]:
args = TrainingArguments('/kaggle/working/outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, dds=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train();

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028979,0.790837


In [37]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds, 0, 1)
preds

array([[0.58880681],
       [0.72489321],
       [0.54129243],
       [0.34883136],
       [0.        ],
       [0.53917426],
       [0.47915918],
       [0.        ],
       [0.18503328],
       [1.        ],
       [0.27734202],
       [0.20898744],
       [0.78518146],
       [0.8414408 ],
       [0.76063174],
       [0.4243502 ],
       [0.25917843],
       [0.        ],
       [0.67042923],
       [0.36464292],
       [0.4476817 ],
       [0.26090789],
       [0.12815706],
       [0.21017854],
       [0.55864877],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.57474762],
       [0.34949943],
       [0.00884697],
       [0.66889763],
       [0.4972114 ],
       [0.43874061],
       [0.15540196]])